In [1]:
#!/usr/bin/env python3
"""Functional Python Programming

Chapter 1, Example Set 1
"""

def sum_numeric():
    """Purely numeric.

    >>> sum_numeric()
    23
    """
    s = 0
    for n in range(1, 10):
        if n % 3 == 0 or n % 5 == 0:
            s += n
    print(s)

def sum_object_light():
    """Some Object Features.

    >>> sum_object_light()
    23
    """
    m = list()
    for n in range(1, 10):
        if n % 3 == 0 or n % 5 == 0:
            m.append(n)
    print(sum(m))

class Summable_List(list):
    def sum(self):
        s = 0
        for v in self:
            s += v
        return s

def sum_full_oo():
    """Full-on OO.

    >>> sum_full_oo()
    23
    """
    m = Summable_List()
    for n in range(1, 10):
        if n % 3 == 0 or n % 5 == 0:
            m.append(n)
    print(m.sum())

def foldr(seq, op, init):
    """Recursive sum.

    >>> foldr( [2,3,5,7], lambda x,y: x+y, 0 )
    17
    """
    if len(seq) == 0:
        return init
    return op(seq[0], sum(seq[1:]))

def until(n, filter_func, v):
    """Build a list: list( filter( filter_func, range(n) ) )

    >>> list( filter( lambda x: x%3==0 or x%5==0, range(10) ) )
    [0, 3, 5, 6, 9]
    >>> until(10, lambda x: x%3==0 or x%5==0, 0)
    [0, 3, 5, 6, 9]
    """
    if v == n:
        return []
    if filter_func(v):
        return [v] + until(n, filter_func, v+1)
    else:
        return until(n, filter_func, v+1)

def sum_functional():
    """
    >>> sum_functional()
    23
    """
    mult_3_5 = lambda x: x%3 == 0 or x%5 == 0
    add = lambda x, y: x+y
    return foldr(until(10, mult_3_5, 0), add, 0)

def sum_hybrid():
    """Hybrid Function.

    >>> sum_hybrid()
    23
    """
    print(sum(n for n in range(1, 10) if n%3 == 0 or n%5 == 0))

def folding():
    """Performance differences from folding.

    >>> ((([]+[1])+[2])+[3])+[4]
    [1, 2, 3, 4]
    >>> []+([1]+([2]+([3]+[4])))
    [1, 2, 3, 4]
    """
    print("foldl", timeit.timeit("((([]+[1])+[2])+[3])+[4]"))
    print("foldr", timeit.timeit("[]+([1]+([2]+([3]+[4])))"))

demo_1 = """
>>> def sumr(seq): 
...     if len(seq) == 0: return 0 
...     return seq[0] + sumr(seq[1:])
>>> sumr([7, 11])
18
>>> sumr([11])
11
>>> sumr([])
0
"""

__test__ = {
    'demo_1': demo_1
}

def test():
    import doctest
    doctest.testmod(verbose=1)

if __name__ == "__main__":
    test()
    # import timeit
    # folding()

Trying:
    def sumr(seq): 
        if len(seq) == 0: return 0 
        return seq[0] + sumr(seq[1:])
Expecting nothing
ok
Trying:
    sumr([7, 11])
Expecting:
    18
ok
Trying:
    sumr([11])
Expecting:
    11
ok
Trying:
    sumr([])
Expecting:
    0
ok
Trying:
    ((([]+[1])+[2])+[3])+[4]
Expecting:
    [1, 2, 3, 4]
ok
Trying:
    []+([1]+([2]+([3]+[4])))
Expecting:
    [1, 2, 3, 4]
ok
Trying:
    foldr( [2,3,5,7], lambda x,y: x+y, 0 )
Expecting:
    17
ok
Trying:
    sum_full_oo()
Expecting:
    23
ok
Trying:
    sum_functional()
Expecting:
    23
ok
Trying:
    sum_hybrid()
Expecting:
    23
ok
Trying:
    sum_numeric()
Expecting:
    23
ok
Trying:
    sum_object_light()
Expecting:
    23
ok
Trying:
    list( filter( lambda x: x%3==0 or x%5==0, range(10) ) )
Expecting:
    [0, 3, 5, 6, 9]
ok
Trying:
    until(10, lambda x: x%3==0 or x%5==0, 0)
Expecting:
    [0, 3, 5, 6, 9]
ok
4 items had no tests:
    __main__
    __main__.Summable_List
    __main__.Summable_List.sum
    __main__.

In [2]:
#!/usr/bin/env python3
"""Functional Python Programming

Chapter 1, Example Set 2

Newton-Raphson root-finding via bisection.

http://www.cs.kent.ac.uk/people/staff/dat/miranda/whyfp90.pdf

Translated from Miranda to Python.
"""
from typing import Callable, Iterator

# next_ = lambda n, x: (x+n/x)/2

def next_(n: float, x: float) -> float:
    # pylint: disable=anomalous-backslash-in-string
    """
    ..  math::

        a_{i+1} = (a_i+n/a_i)/2

    Converges on

    ..  math::

        a = (a+n/a)/2

    So

    ..  math::

        2a  &= a+n/a \\
        a   &= n/a \\
        a^2 &= n \\
        a   &= \sqrt n
    """
    return (x+n/x)/2

def repeat(f: Callable[[float], float], a: float) -> Iterator[float]:
    """yields a, f(a), f(f(a)), etc."""
    yield a
    yield from repeat(f, f(a))

def within(eps: float, iterable: Iterator[float]) -> Iterator[float]:
    def head_tail(eps: float, a: float, iterable: Iterator[float]):
        b = next(iterable)
        if abs(a-b) <= eps:
            return b
        return head_tail(eps, b, iterable)

    return head_tail(eps, next(iterable), iterable)

def sqrt(a0: float, eps: float, n: float):
    return within(eps, repeat(lambda x: next_(n, x), a0))

def test():
    """
    >>> round(next_( 2, 1.5 ), 4)
    1.4167
    >>> n= 2
    >>> f= lambda x: next_( n, x )
    >>> a0= 1.0
    >>> [ round(x,4) for x in (a0, f(a0), f(f(a0)), f(f(f(a0))),) ]
    [1.0, 1.5, 1.4167, 1.4142]

    >>> within( .5, iter([3, 2, 1, .5, .25]) )
    0.5

    >>> round( sqrt( 1.0, .0001, 3 ), 6 )
    1.732051
    >>> round(1.732051**2, 5)
    3.0
    """
    import doctest
    doctest.testmod(verbose=1)

if __name__ == "__main__":
    test()


Trying:
    def sumr(seq): 
        if len(seq) == 0: return 0 
        return seq[0] + sumr(seq[1:])
Expecting nothing
ok
Trying:
    sumr([7, 11])
Expecting:
    18
ok
Trying:
    sumr([11])
Expecting:
    11
ok
Trying:
    sumr([])
Expecting:
    0
ok
Trying:
    ((([]+[1])+[2])+[3])+[4]
Expecting:
    [1, 2, 3, 4]
ok
Trying:
    []+([1]+([2]+([3]+[4])))
Expecting:
    [1, 2, 3, 4]
ok
Trying:
    foldr( [2,3,5,7], lambda x,y: x+y, 0 )
Expecting:
    17
ok
Trying:
    sum_full_oo()
Expecting:
    23
ok
Trying:
    sum_functional()
Expecting:
    23
ok
Trying:
    sum_hybrid()
Expecting:
    23
ok
Trying:
    sum_numeric()
Expecting:
    23
ok
Trying:
    sum_object_light()
Expecting:
    23
ok
Trying:
    round(next_( 2, 1.5 ), 4)
Expecting:
    1.4167
ok
Trying:
    n= 2
Expecting nothing
ok
Trying:
    f= lambda x: next_( n, x )
Expecting nothing
ok
Trying:
    a0= 1.0
Expecting nothing
ok
Trying:
    [ round(x,4) for x in (a0, f(a0), f(f(a0)), f(f(f(a0))),) ]
Expecting:
    [1.0, 

In [3]:
#!/usr/bin/env python3
"""Functional Python Programming

Chapter 2, Example Set 1
"""
# pylint: disable=missing-docstring,wrong-import-position

from typing import Iterator
def numbers() -> Iterator[int]:
    for i in range(1024):
        # print(f"= {i}")
        yield i

def sum_to(n: int) -> int:
    sum: int = 0
    for i in numbers():
        if i == n:
            break
        sum += i
    return sum


def namedtuples():
    """nametuple vs. class performance"""
    import timeit
    class_time = timeit.timeit(
        """x= X(1,2,3)""",
        """
class X:
    def __init__( self, a, b, c ):
        self.a= a
        self.b= b
        self.c= c
        """)
    print(f"class {class_time}")

    tuple_time = timeit.timeit("""x= (1,2,3)""")
    print(f"tuple {tuple_time}")

    collections_nt_time = timeit.timeit(
        """x= X(1,2,3)""",
        """
from collections import namedtuple
X = namedtuple( "X", ("a", "b", "c") )
        """)
    print(f"namedtuple {collections_nt_time}")

    typing_nt_time = timeit.timeit(
        """x= X(1,2,3)""",
        """
from typing import NamedTuple
class X(NamedTuple):
    a: str
    b: str
    c: str
        """)
    print(f"NamedTuple {typing_nt_time}")

import math
def isprimei(n: int) -> bool:
    """Is n prime?

    >>> isprimei(2)
    True
    >>> tuple( isprimei(x) for x in range(3,11) )
    (True, False, True, False, True, False, False, False)
    """
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, 1+int(math.sqrt(n)), 2):
        if n % i == 0:
            return False
    return True

def isprimer(n: int) -> bool:
    """Is n prime?

    >>> isprimer(2)
    True
    >>> tuple( isprimer(x) for x in range(3,11) )
    (True, False, True, False, True, False, False, False)
    """
    def isprime(k: int, coprime: int) -> bool:
        """Is k relatively prime to the value coprime?"""
        if k < coprime*coprime:
            return True
        if k % coprime == 0:
            return False
        return isprime(k, coprime+2)

    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    return isprime(n, 3)

def isprimeg(n: int) -> bool:
    """Is n prime?

    >>> isprimeg(2)
    True
    >>> tuple( isprimeg(x) for x in range(3,11) )
    (True, False, True, False, True, False, False, False)

    Remarkably slow for large primes, for example, M_61=2**61-1.

    >>> isprimeg(62710593)
    False
    """
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    return not any(n%p == 0 for p in range(3, int(math.sqrt(n))+1, 2))

def recursion():
    """Recursion Performance Comparison.
    """
    import timeit
    print("isprimei",
          timeit.timeit(
              """isprimei(131071)""",
              """
import math
def isprimei( n ):
    if n < 2: return False
    if n == 2: return True
    if n % 2 == 0: return False
    for i in range(3,1+int(math.sqrt(n)),2):
        if n % i == 0:
            return False
    return True
              """, number=100000))

    print("isprimer",
          timeit.timeit(
              """isprimer(131071)""",
              """
def isprimer( n ):
    def isprime( n, coprime ):
        if n < coprime*coprime: return True
        if n % coprime == 0: return False
        return isprime( n, coprime+2 )

    if n < 2: return False
    if n == 2: return True
    if n % 2 == 0: return False
    return isprime( n, 3 )
              """, number=100000))

    print("isprimeg",
          timeit.timeit(
              """isprimeg(131071)""",
              """
import math
def isprimeg( n ):
    if n < 2: return False
    if n == 2: return True
    if n % 2 == 0: return False
    return not any(n%p==0 for p in range(3,int(math.sqrt(n))+2))
              """, number=100000))

def limit_of_performance():
    """We can see that testing a large prime is
    quite slow. Testing large non-primes is quite fast.
    """
    import time

    t = time.perf_counter()
    for i in range(30, 89):
        m = 2**i-1
        print(i, m, end="")
        if isprimeg(m):
            print("prime", end="")
        else:
            print("composite", end="")
        print(time.perf_counter() - t)


__test__ = {
    'higher_order':
    """Higher Order Functions.

>>> year_cheese = [(2000, 29.87), (2001, 30.12), (2002, 30.6),
... (2003, 30.66), (2004, 31.33), (2005, 32.62), (2006, 32.73),
... (2007, 33.5), (2008, 32.84), (2009, 33.02), (2010, 32.92)
... ]

>>> max( year_cheese )
(2010, 32.92)
>>> max( year_cheese, key= lambda yc: yc[1] )
(2007, 33.5)

Wrap-prcess-unwrap
>>> max(map(lambda yc: (yc[1], yc), year_cheese))[1]
(2007, 33.5)

>>> wrapped = map(lambda yc: (yc[1], yc), year_cheese)
>>> processed = max(wrapped)
>>> processed[1]
(2007, 33.5)

>>> snd= lambda x: x[1]
>>> snd(max(map(lambda yc: (yc[1],yc), year_cheese)))
(2007, 33.5)

    """,
    'sum_non_strict':
    """Non-strict generators.

>>> sum_to(5)
10
    """,
    'other tests':
    """
>>> def example(a, b, **kw):
...     return a*b
...
>>> type(example)
<class 'function'>
>>> example.__code__.co_varnames
('a', 'b', 'kw')
>>> example.__code__.co_argcount
2
>>> isprimei(131071)
True
>>> isprimer(131071)
True
>>> isprimeg(131071)
True
    """
    }

def test():
    import doctest
    doctest.testmod(verbose=2)

if __name__ == "__main__":
    test()
    namedtuples()
    #recursion()
    #limit_of_performance()


Trying:
    year_cheese = [(2000, 29.87), (2001, 30.12), (2002, 30.6),
    (2003, 30.66), (2004, 31.33), (2005, 32.62), (2006, 32.73),
    (2007, 33.5), (2008, 32.84), (2009, 33.02), (2010, 32.92)
    ]
Expecting nothing
ok
Trying:
    max( year_cheese )
Expecting:
    (2010, 32.92)
ok
Trying:
    max( year_cheese, key= lambda yc: yc[1] )
Expecting:
    (2007, 33.5)
ok
Trying:
    max(map(lambda yc: (yc[1], yc), year_cheese))[1]
Expecting:
    (2007, 33.5)
ok
Trying:
    wrapped = map(lambda yc: (yc[1], yc), year_cheese)
Expecting nothing
ok
Trying:
    processed = max(wrapped)
Expecting nothing
ok
Trying:
    processed[1]
Expecting:
    (2007, 33.5)
ok
Trying:
    snd= lambda x: x[1]
Expecting nothing
ok
Trying:
    snd(max(map(lambda yc: (yc[1],yc), year_cheese)))
Expecting:
    (2007, 33.5)
ok
Trying:
    def example(a, b, **kw):
        return a*b
Expecting nothing
ok
Trying:
    type(example)
Expecting:
    <class 'function'>
ok
Trying:
    example.__code__.co_varnames
Expecting:


In [4]:
#!/usr/bin/env python3
"""Functional Python Programming

Chapter 3, Example Set 1
"""
from typing import Callable

class Mersenne1:
    """Callable object with a **Strategy** plug in required."""
    def __init__(self, algorithm: Callable[[int], int]) -> None:
        self.pow2 = algorithm
    def __call__(self, arg: int) -> int:
        return self.pow2(arg)-1

def shifty(b: int) -> int:
    """2**b via shifting.

    >>> shifty(17)-1
    131071
    """
    return 1 << b

def multy(b: int) -> int:
    """2**b via naive recursion.

    >>> multy(17)-1
    131071
    """
    if b == 0:
        return 1
    return 2*multy(b-1)

def faster(b: int) -> int:
    """2**b via faster divide-and-conquer recursion.

    >>> faster(17)-1
    131071
    """
    if b == 0:
        return 1
    if b%2 == 1:
        return 2*faster(b-1)
    t = faster(b//2)
    return t*t

# Implementations of Mersenne with strategy objects plugged in properly.

m1s = Mersenne1(shifty)
m1m = Mersenne1(multy)
m1f = Mersenne1(faster)

# Alternative Mersenne using class-level configuration.
# The syntax is awkward.

class Mersenne2:
    pow2: Callable[[int], int] = None
    def __call__(self, arg: int) -> int:
        pow2 = self.__class__.__dict__['pow2']
        return pow2(arg)-1

class ShiftyMersenne(Mersenne2):
    pow2 = shifty

class MultyMersenee(Mersenne2):
    pow2 = multy

class FasterMersenne(Mersenne2):
    pow2 = faster

m2s = ShiftyMersenne()
m2m = MultyMersenee()
m2f = FasterMersenne()

test_mersenne = """
>>> m1s(17)
131071
>>> m1m(17)
131071
>>> m1f(17)
131071
>>> m2s(17)
131071
>>> m2m(17)
131071
>>> m2f(17)
131071
>>> m1s(89)
618970019642690137449562111
>>> m1m(89)
618970019642690137449562111
>>> m1f(89)
618970019642690137449562111
"""

test_pure = """
>>> def m(n):
...     return 2**n-1
>>> m(89)
618970019642690137449562111
"""

__test__ = {
    'test_mersenne': test_mersenne,
    'test_pure': test_pure
}
def test():
    import doctest
    doctest.testmod(verbose=2)

def performance():
    import timeit
    print(m1s.pow2.__name__,
          timeit.timeit(
              """m1s(17)""",
              """from Chapter_3.ch03_ex1 import m1s"""))
    print(m1m.pow2.__name__,
          timeit.timeit(
              """m1m(17)""",
              """from Chapter_3.ch03_ex1 import m1m"""))
    print(m1f.pow2.__name__,
          timeit.timeit(
              """m1f(17)""",
              """from Chapter_3.ch03_ex1 import m1f"""))

if __name__ == "__main__":
    import sys
    print(sys.version)
    test()
    # import timeit
    # performance()


3.6.2 |Continuum Analytics, Inc.| (default, Jul 20 2017, 12:30:02) [MSC v.1900 64 bit (AMD64)]
Trying:
    m1s(17)
Expecting:
    131071
ok
Trying:
    m1m(17)
Expecting:
    131071
ok
Trying:
    m1f(17)
Expecting:
    131071
ok
Trying:
    m2s(17)
Expecting:
    131071
ok
Trying:
    m2m(17)
Expecting:
    131071
ok
Trying:
    m2f(17)
Expecting:
    131071
ok
Trying:
    m1s(89)
Expecting:
    618970019642690137449562111
ok
Trying:
    m1m(89)
Expecting:
    618970019642690137449562111
ok
Trying:
    m1f(89)
Expecting:
    618970019642690137449562111
ok
Trying:
    def m(n):
        return 2**n-1
Expecting nothing
ok
Trying:
    m(89)
Expecting:
    618970019642690137449562111
ok
Trying:
    faster(17)-1
Expecting:
    131071
ok
Trying:
    ((([]+[1])+[2])+[3])+[4]
Expecting:
    [1, 2, 3, 4]
ok
Trying:
    []+([1]+([2]+([3]+[4])))
Expecting:
    [1, 2, 3, 4]
ok
Trying:
    foldr( [2,3,5,7], lambda x,y: x+y, 0 )
Expecting:
    17
ok
Trying:
    isprimeg(2)
Expecting:
    True
ok
Try